In [105]:
import numpy as np
import random
import ciw
import collections

# Define the model

In [14]:
def build_model(lambda_a, lambda_o, mu, total_capacity):
    model = ciw.create_network(
        arrival_distributions = [ciw.dists.Exponential(lambda_a),
                             ciw.dists.Exponential(lambda_o)],
        service_distributions = [ciw.dists.Deterministic(0),
                                 ciw.dists.Exponential(mu)],
        routing=[[0.0, 1.0],
                 [0.0, 0.0]],
        number_of_servers=[float('inf'), total_capacity]
    )
    return model

# Inherit 'ciw.Node' class to block individuals of Node 1

In [15]:
def build_custom_node(threshold=7):
    class CustomNode(ciw.Node):
        def finish_service(self):
            """
            The next individual finishes service:
              - finds the individual to finish service
              - check if they need to change class
              - find their next node
              - release the individual if there is capacity at destination,
                otherwise cause blockage
            """
            next_individual, next_individual_index = self.find_next_individual()
            self.change_customer_class(next_individual)
            next_node = self.next_node(next_individual)
            next_individual.destination = next_node.id_number

            if not np.isinf(self.c):
                next_individual.server.next_end_service_date = float('Inf')

            blockage = (next_node.number_of_individuals >= threshold and self.id_number == 1)
            if (next_node.number_of_individuals < next_node.node_capacity) and not blockage:
                self.release(next_individual_index, next_node)
            else:
                self.block_individual(next_individual, next_node)
                
    return CustomNode

# Simulate the problem

In [16]:
def simulate_model(lambda_a, lambda_o, mu, total_capacity, threshold, seed_num = random.random()):
    
    model = build_model(lambda_a, lambda_o, mu, total_capacity)
    node = build_custom_node(threshold)
    
    ciw.seed(seed_num)
    
    Simulation = ciw.Simulation(model, node_class=node)
    Simulation.simulate_until_max_time(1440)
    
    return Simulation

# Trials

In [ ]:
for trial in range(10):
    ciw.seed(trial)
    Q = ciw.Simulation(N)
    Q.simulate_until_max_time(600)
    recs = Q.get_all_records()
    num_completed = len([r for r in recs if r.node == 3 and r.arrival_date < 500])
    completed_custs.append(num_completed)

In [110]:
# def multiple_runs(num_of_trials):
num_of_trials = 10
warm_up_time = 100

waits = []
services = []
blocks = []
for trial in range(num_of_trials):
    simulation = simulate_model(lambda_a, lambda_o, mu, total_capacity, threshold, trial)
    sim_results = simulation.get_all_records()
    
    waiting = [r.waiting_time for r in sim_results if r.arrival_date > warm_up_time]
    serving = [r.service_time for r in sim_results if r.arrival_date > warm_up_time]
    blocking = [r.time_blocked for r in sim_results if r.arrival_date > warm_up_time]
    waits.append(waiting)
    services.append(serving)
    blocks.append(blocking)
    

In [113]:
records = collections.namedtuple('records', 'waiting_times service_times blocking_times')
results = records(waits, services, blocks)

print(np.mean([np.mean(w) for w in results.waiting_times]))
print(np.mean([np.mean(s) for s in results.service_times]))
print(np.mean([np.mean(b) for b in results.blocking_times]))

print(np.mean([np.mean(w) for w in waits]))
print(np.mean([np.mean(s) for s in services]))
print(np.mean([np.mean(b) for b in blocks]))

# Results of the model

In [201]:
lambda_a = 0.15
lambda_o = 0.2
mu = 0.05
total_capacity = 8
threshold = 4

In [202]:
simulation = simulate_model(lambda_a, lambda_o, mu, total_capacity, threshold, 1)
sim_results = simulation.get_all_records()

In [203]:
# Mean Waiting Time
waits = [r.waiting_time for r in sim_results]
mean_waiting_time = sum(waits) / len(waits)
mean_waiting_time

2.9666802537799954

In [204]:
# Services in each node
service_nodes = [r.node for r in sim_results]
collections.Counter(service_nodes)

Counter({1: 181, 2: 466})

In [205]:
# Mean Time Blocked
Blocks = [r.time_blocked for r in sim_results]
mean_time_blocked = sum(Blocks) / len(Blocks)
mean_time_blocked

1.401489260432193

# Random Examples for tests

In [231]:
ciw.seed(5)
Q = ciw.Simulation(build_model(1, 1, 2, 1))

Q.simulate_until_max_time(100)
records = Q.get_all_records()
waits = [r.waiting_time for r in records]
blocks = [r.time_blocked for r in records]

print(len(records))
print(sum(waits))
print(sum(blocks))

290
1089.854729732795
0.0


In [233]:
ciw.seed(5)
Q = ciw.Simulation(build_model(1, 1, 2, 1), node_class=build_custom_node(7))

Q.simulate_until_max_time(100)
records = Q.get_all_records()
waits = [r.waiting_time for r in records]
blocks = [r.time_blocked for r in records]

print(len(records))
print(sum(waits))
print(sum(blocks))

290
1026.2910789050652
66.03415121579033


In [260]:
sim_results = []
for i in range(10):
    simulation = simulate_model(0.15, 0.2, 0.05, 8, 4, i)
    sim_results.append(len(simulation.get_all_records()))

In [261]:
sim_results

[705, 647, 712, 747, 681, 738, 699, 724, 744, 700]